In [ ]:
# README

### This file is intended for use as supplementary material for the NeurIPS 2019 submission
### "High Dimensional Causal Discovery". It acts as a wrapper for the several inference
### algorithms that are discussed in the paper. Its primary purpose is to serve as an 
### oracle for the numerical results presented in Section 4 of the paper. All data 
### generation used for testing is computed here, so that the examined causal inference
### algorithms can be objectively analyzed and compared. Some of the data used for our 
### zebrafish results (Section 5) is proprietary, and the remainder requires more space 
### than allowed, however, the data that has been made publically available by its
### curators exists here:

### https://janelia.figshare.com/articles/Whole-brain_light-sheet_imaging_data/7272617


### DEPENDENCIES

### This Jupyter notebook requires an array of dependencies in order to fully function 
### without additional modifications:

### A Unix-based OS (MacOS, Linux, etc.) 

### Python3 Libraries: numpy, scipy, sklearn, joblib, matplotlib, networkx, pickle, dill,
### sortedcontainers, argparse, rpy2

### In order to run the algorithms implemented in causal-cmd/Tetrad, the current up-to-date
### version of the software can be found here:

### https://cloud.ccd.pitt.edu/nexus/content/repositories/releases/edu/pitt/dbmi/causal-cmd/1.0.0/

### Put the file "causal-cmd-1.0.0-jar-with-dependencies.jar" in the same directory as this
### file.

### Additionally, in order to run the BigQUIC algorithm from this notebook, an up-to-date  
### version of R, linked to rpy2, is also required. You must also run the following line
### of code from the R console:

### install.packages('BigQuic', repos='http://cran.us.r-project.org')

### We did not ultimately include GSP in this notebook due to RAM explosion issues during runtime 
### in the Jupyter notebook, however the current up-to-date version exists here:

### https://github.com/uhlerlab/causaldag

### The fges-py implementation, as well as the causal-cmd version have been included in the supplementary material
### alongside this document.
### We do not claim authorship of causal-cmd, and have merely included it for reader convenience.

In [1]:
import numpy as np
import joblib
import scipy.io
import random
import os
import runner
import time
import rpy2
from rpy2.robjects.packages import importr
import rpy2.robjects.numpy2ri as numpy2ri
from sklearn.covariance import GraphLasso

R = rpy2.robjects.r
importr('BigQuic')

def generate_random_dag(nnodes, avg_degree):
    '''Generate a random dag with nnodes nodes and an average degree of avg_degree'''
    graph = []
    num_edges = float((nnodes * avg_degree) / 2)
    tot_edges = float(nnodes*(nnodes-1)/2)   
    for i in range(nnodes):
        for j in range(i+1, nnodes):
            if tot_edges > 0:
                odds = float(num_edges / tot_edges)
                if random.random() < odds:
                    graph.append((i,j))
                    num_edges -= 1
                tot_edges -= 1
    return graph

dag = generate_random_dag(10, 2)

def generate_data_from_dag(dag, nnodes, sample_size):
    '''Given a dag generated by the previous function, generate randomized Gaussian 
    data from this dag with sample size sample_size'''
    dag_dict = {}
    parents = {}
    for edge in dag:
        dag_dict[edge] = random.uniform(0.5,0.7)
    for i in range(nnodes):
        for j in dag:
            if j[0] == i:
                if j[1] not in parents:
                    parents[j[1]] = {i}
                else:
                    parents[j[1]].update({i})
    for i in range(nnodes):
        if i not in parents:
            parents[i] = {}
    result = np.zeros((sample_size, nnodes))
    for i in range(sample_size):
        completed_nodes = set()
        samples = np.zeros(nnodes)
        for k in range(nnodes):
            if parents[k] == {}:
                samples[k] = np.random.normal()
                completed_nodes.add(k)
        while (not completed_nodes == set(list(range(nnodes)))):
            for j in parents:
                if j not in completed_nodes:
                    if set(parents[j]).issubset(completed_nodes):
                        tot = 0
                        for par in parents[j]:
                            tot += dag_dict[(par, j)] * samples[par]
                        tot +=  np.random.normal()
                        samples[j] = tot
                        completed_nodes.add(j)
        result[i] = samples
    np.savetxt('test.tmp' ,result)
    return result, dag_dict

def compare_graphs_directed(g1, g2):
    '''Compute the precision and recall of g1 compared to the ground truth g2
    For directed methods only'''
    true_positive = 0
    false_positive = 0
    false_negative = 0
    for i in g1:
        if i in g2:
            true_positive += 1
        else:
            false_positive += 1
    for i in g2:
        if i not in g1:
            false_negative += 1
    return (true_positive / (true_positive + false_positive), 
            true_positive / (true_positive + false_negative))

def compare_graphs_undirected(g1, g2):
    '''Compute the precision and recall of g1 compared to the ground truth g2
    For undirected methods only'''
    true_positive = 0
    false_positive = 0
    false_negative = 0
    for (i1, i2) in g1:
        if (i1, i2) in g2 or (i2, i1) in g2:
            true_positive += 1
        else:
            false_positive += 1
    for (i1, i2) in g2:
        if (i1, i2) not in g1 and (i2, i1) not in g1:
            false_negative += 1
    return (true_positive / (true_positive + false_positive), 
            true_positive / (true_positive + false_negative))

def get_graph_from_ccmd_text(filename):
    '''Extract graph from causal-cmd output into Python-readable format'''
    f = open(filename, 'r') 
    while f.readline()[:11] != 'Graph Edges':
        continue
    edge = '0'
    edges  = []
    # Just string parsing
    while edge != '' and str(edge[0])[0] in '0123456789':
        edge = (f.readline())
        lst = edge.split()
        if lst != []:
            edge = (int(lst[1][1:]) - 1, int(lst[3][1:]) - 1)
            edges.append(edge)
    return edges

def extract_undir_graph_from_mat(mat, num_edges):
    '''Convert a precision matrix into an undirected graph with num_edges edges
    by performing binary search over inverse covariance thresholds'''
    pres = np.abs(mat)
    # Set the bottom triangle of the matrix to zero to prevent repetition
    for i in range(len(pres)):
        for j in range(len(pres)):
            if i >= j:
                pres[i][j] = 0
    # Perform binary search on covariance thresholding to determine cutoff value
    cutoff = 5
    split = 2.5
    while True:
        num = np.sum(pres > cutoff)
        if split < 0.000001:
            break
        elif num > num_edges:
            cutoff += split
            split /= 2
        elif num < num_edges:
            cutoff -= split
            split /= 2
        else: 
            break
    thresh_mat = 1*(pres > cutoff)
    edges = []
    for ei, i in enumerate(thresh_mat):
        for ej, j in enumerate(i):
            if j:
                edges.append((ei,ej))
    return edges


def bigquic(mat, alpha):
    """Wrapper for BigQUIC"""
    r_mat =  numpy2ri.numpy2rpy(mat) 
    # Run the R program
    # Source: https://github.com/gregversteeg/py_bigquic
    program_string = 'f <- function(r) {out = BigQuic(X=r, lambda=' + str(alpha) + ', use_ram=TRUE);'
    program_string += 'as(out$precision_matrices[[1]], "matrix")}'
    f = R(program_string)
    prec = np.array(f(r_mat))
    return prec

def test_alg(algorithm='fges-py', nnodes=1000, avg_degree=5, penalty=5, sample_size=1000, alpha=0.01):
    '''
    Test any algorithm of your choice, varying the hyperparameters:
    nnodes: number of nodes in the generated dag
    avg_degree: Average degree of the generated dag
    sample_oze: Sample size of the data generated from the dag
    penalty: Sparsity penalty. Only applies to some algorithms. (fges,fask,pc)
    alpha: Regularization term. Only applies to some algorithms. (gLasso, BigQUIC)
    '''
    dag = generate_random_dag(nnodes, avg_degree)
    data = generate_data_from_dag(dag, nnodes, sample_size)[0]
    time1 = time.time()
    print("Algorithm Starting")
    if algorithm == 'fges-py':
        runner.main(dataset="test.tmp",save_name="fges_results", sparsity=penalty)
        result = list(joblib.load("fges_results.pkl")['graph'].edges)
        directed = True
    elif algorithm == 'causal-cmd-fges':
        os.system('java -jar causal-cmd-1.0.0-jar-with-dependencies.jar  --algorithm fges --data-type continuous --dataset test.tmp --delimiter space --score sem-bic --skip-latest --penaltyDiscount ' + str(float(penalty)) + ' --no-header --prefix results')
        result = get_graph_from_ccmd_text('results.txt')
        directed = True
    elif algorithm == 'fask':
        os.system('java -jar causal-cmd-1.0.0-jar-with-dependencies.jar  --algorithm fask --data-type continuous --dataset test.tmp --delimiter space --score sem-bic --skip-latest --penaltyDiscount ' + str(float(penalty)) + ' --no-header --useFasAdjacencies --prefix results')
        result = get_graph_from_ccmd_text('results.txt')
        directed = True
    elif algorithm == 'heut-pc':
        os.system('java -jar causal-cmd-1.0.0-jar-with-dependencies.jar --algorithm pc-all --data-type continuous --dataset test.tmp --delimiter space --test sem-bic --skip-latest --penaltyDiscount ' + str(float(penalty)) + ' --no-header --prefix results --stableFAS')
        result = get_graph_from_ccmd_text('results.txt')
        directed = True
    elif algorithm == 'bigquic-R':
        prec_mat = bigquic(data, alpha=alpha)
        result = extract_undir_graph_from_mat(prec_mat, (nnodes * avg_degree) / 2)
        directed = False
    elif algorithm == 'glasso-sklearn':
        model = GraphLasso(alpha=alpha)
        data -= data.mean(axis=0)
        stds = np.std(data, axis=0).clip(1e-10)
        data /= stds
        model.fit(data)
        prec_mat = model.precision_
        result = extract_undir_graph_from_mat(prec_mat, (nnodes * avg_degree) / 2)
        directed = False
    time2 = time.time()
    if directed:
        precision, recall = compare_graphs_directed(result, dag)
    else:
        precision, recall = compare_graphs_undirected(result, dag)
    os.system('rm test.tmp causal-cmd.log results.txt fges_results.pkl')
    return precision, recall, (time2 - time1)



In [ ]:
test_alg(algorithm='fges-py')